In [8]:
import pandas as pd
import pvlib

FILE_IN = r'psm3_demo.csv'
FILE_OUT = r'psm3_pytest.csv'

In [9]:
header = []
with open(FILE_IN, 'r') as file:
    header.append(file.readline())
    header.append(file.readline())

data, meta = pvlib.iotools.read_psm3(FILE_IN)

In [15]:
dni = data.DNI
dhi = data.DHI
ghi = data.GHI
time_range = data.index

solar_pos = pvlib.solarposition.get_solarposition(time=time_range, latitude=meta['Latitude'],
                                            longitude=meta['Longitude'], altitude=meta['Elevation'])

poa_df = pvlib.irradiance.get_total_irradiance(surface_tilt=20, surface_azimuth=180, 
                                                    solar_zenith=solar_pos.apparent_zenith,
                                                    solar_azimuth=solar_pos.azimuth,
                                                    dni=dni, ghi=ghi, dhi=dhi)
poa_global = poa_df.poa_global

In [11]:
cell_params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']

temp_cell = pvlib.temperature.sapm_cell(poa_global=poa_global, temp_air=data['Temperature'],
                                        wind_speed=data['Wind Speed'], **cell_params)

In [12]:
temp_module = pvlib.temperature.sapm_module(poa_global=poa_global, temp_air=data['Temperature'],
                                            wind_speed=data['Wind Speed'], a=cell_params['a'],
                                            b=cell_params['b'])

In [13]:
data['poa_global'] = poa_global
data['temp_cell'] = temp_cell
data['temp_module'] = temp_module

In [14]:
with open(FILE_OUT, 'w') as file:
    for line in header:
        file.write(line)
    data.to_csv(file, index=False)